In [1]:
from utilities.azure_cosmos import init_container

container = init_container()

In [2]:
import sys
sys.path.append('..')  # 将父文件夹添加到 Python 路径中，不然找不到下面的包

from utilities.tushare_limit_up import generate_limit_up_df

trade_date = "20240313"

df, cal_date = generate_limit_up_df(trade_date)

In [9]:
# prepare data
import numpy as np

df['partitionKey'] = trade_date
df['id'] = trade_date + "_" + df['ts_code']
df = df.replace({np.nan: None})

df

,ts_code,name,industry,close,pct_chg,amount,circ_mv,total_mv,pb,pe,...,turnover_rate_f,limit_amount,fd_amount,first_time,last_time,open_times,up_stat,limit_times,partitionKey,id
52,603261.SH,立航科技,航空,45.84,10.0072,6.241033,12.054671,35.901806,4.1119,102.2995,...,52.6227,None,0.283933,93213,144513,6.0,7/7,7.0,20240313,20240313_603261.SH
56,603580.SH,艾艾精工,塑料,18.41,9.9761,2.146402,24.056936,24.056936,5.4561,None,...,27.1526,None,0.500989,93017,93814,2.0,7/7,7.0,20240313,20240313_603580.SH
13,002298.SZ,中电兴发,软件服务,6.36,10.0346,0.501243,40.673719,47.071053,1.1453,None,...,1.261,None,3.617805,92500,92500,0.0,4/4,4.0,20240313,20240313_002298.SZ
25,002902.SZ,铭普光磁,通信设备,34.43,10.0000,21.767576,51.880130,73.136626,6.8972,106.1832,...,48.5507,None,0.031387,92500,150000,5.0,4/4,4.0,20240313,20240313_002902.SZ
55,603533.SH,掌阅科技,互联网,24.90,9.9823,12.825755,109.285312,109.285312,4.2894,189.8067,...,24.7655,None,0.995552,94457,110739,5.0,3/3,3.0,20240313,20240313_603533.SH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,300900.SZ,广联航空,航空,24.77,13.1050,4.493151,38.457901,52.452301,3.7258,34.9978,...,13.2131,None,None,None,None,None,None,None,20240313,20240313_300900.SZ
37,301285.SZ,鸿日达,元器件,26.66,14.6174,2.590702,20.584135,55.098222,5.188,111.7264,...,13.3875,None,None,None,None,None,None,None,20240313,20240313_301285.SZ
38,301588.SZ,N美新,家居用品,37.39,157.8621,6.608119,8.443597,44.444653,5.0019,58.5028,...,77.6454,None,None,None,None,None,None,None,20240313,20240313_301588.SZ
66,688695.SH,N中创,软件服务,48.65,116.8970,6.693719,8.749734,41.377495,5.2178,62.0693,...,74.0278,None,None,None,None,None,None,None,20240313,20240313_688695.SH


[通过Python创建存储过程](https://learn.microsoft.com/en-us/azure/cosmos-db/nosql/how-to-use-stored-procedures-triggers-udfs?tabs=python-sdk)

In [5]:
from utilities.azure_cosmos import create_stored_procedure
create_stored_procedure(container)

Stored procedure 'bulkInsert' already exists


In [6]:
def query_items(container, partitionKey):
    # Including the partition key value of partitionKey in the WHERE filter results in a more efficient query
    items = list(container.query_items(
        query="SELECT * FROM r WHERE r.partitionKey=@partitionKey",
        parameters=[
            { "name":"@partitionKey", "value": partitionKey }
        ]
    ))
    return items

In [11]:
items = query_items(container, trade_date)

if items:
    print(items)
else:
    documents = df.to_dict('records')

    result = container.scripts.execute_stored_procedure("bulkInsert",params=[documents, trade_date], partition_key=trade_date)
    result